In [1]:
%pip install python-binance

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from binance.client import Client
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Initialize the client
client = Client(api_key='your_api_key', api_secret='your_api_secret')

# Get historical data for BTCUSDT with 4-hour interval
klines = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "1 Jan, 2010", "28 Feb, 2025")

# Create a DataFrame
df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Set timestamp as index
df.set_index('timestamp', inplace=True)

btc = df[['close']].copy()
print(btc.head())


NameError: name 'client' is not defined

In [ ]:
def calculate_rsi(data, window):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Calculate RSI for the btc DataFrame
btc['close'] = btc['close'].astype(float)
btc['rsi'] = calculate_rsi(btc['close'], window=14)
print(btc.head(20))

In [ ]:
def portfolio(rsi_up = 50, rsi_down = 50, hand = 10):
    btc['signal'] = np.where(btc['rsi'] < rsi_down, 1, np.where(btc['rsi'] > rsi_up, -1, 0))

    btc['position'] = hand / btc['close'] * btc['signal']

    btc['total_position'] = 0
    for i in range(1, len(btc)):
        btc['total_position'].iloc[i] = btc['position'].iloc[i] + btc['total_position'].iloc[i-1]

    btc['cost'] = np.where(btc['signal'] == -1, -btc['close'] * btc['position'], np.where(btc['signal'] == 1, -btc['close'] * btc['position'], 0))

    btc['total_cost'] = 0.0

    for i in range(1, len(btc)):
        btc['total_cost'].iloc[i] = btc['cost'].iloc[i] + btc['total_cost'].iloc[i-1]
    
    btc['pnl'] = btc['total_cost'] + (btc['total_position'] * btc['close'])
    btc.to_csv('btc.csv')
    
    return btc[['total_cost', 'pnl']]

portfolio(rsi_up = 10, rsi_down = 100 - 10, hand = 10).iloc[-1]

In [ ]:
result = pd.DataFrame()
for _ in range(0, 110, 10):
    result = pd.concat([result, portfolio(rsi_up = 100 - _, rsi_down = _, hand = 10).iloc[-1].to_frame().T])

result.reset_index(drop=True, inplace=True)
result['total_cost'] = -result['total_cost']
result = result.drop(result.index[0])
result['return'] = result['pnl'] / result['total_cost']

result

In [ ]:
plt.plot(result['pnl'], label='pnl')
plt.xlabel('rsi')
plt.ylabel('pnl')
plt.title('pnl vs rsi')
plt.show()
plt.plot(result['total_cost'], label='total_cost')
plt.xlabel('rsi')
plt.ylabel('total_cost')
plt.title('total_cost vs rsi')
plt.show()
plt.plot(result['return'], label='return')
plt.title('return vs rsi')
plt.xlabel('rsi')
plt.ylabel('return')
plt.show()

In [ ]:
best_rsi_up = result['return'].idxmax() * 10
best_rsi_up